In [5]:
from pyspark import SparkContext

In [6]:
sc = SparkContext(master="local", appName="transformacionesyAcciones")

22/03/27 21:15:50 WARN Utils: Your hostname, luishm resolves to a loopback address: 127.0.1.1; using 192.168.0.117 instead (on interface wlp7s0)
22/03/27 21:15:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/27 21:15:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
rdd1 = sc.parallelize([1,2,3])
type(rdd1)

pyspark.rdd.RDD

In [5]:
rdd1.collect()

[1, 2, 3]

In [8]:
!ls ../files

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [8]:
equiposOlimpicosrdd = sc.textFile("../files/paises.csv") \
                      .map(lambda line : line.split(","))

In [9]:
equiposOlimpicosrdd.take(15)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR'],
 ['10', 'Alcyon-6', 'FRA'],
 ['11', 'Alcyon-7', 'FRA'],
 ['12', 'Aldebaran', 'ITA'],
 ['13', 'Aldebaran II', 'ITA'],
 ['14', 'Aletta', 'IRL']]

In [33]:
equiposOlimpicosrdd.map(lambda x: (x[2])).distinct().count()

231

In [10]:
equiposOlimpicosrdd.map(lambda x: (x[2], x[1])).groupByKey().mapValues(len).take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [11]:
equipos_argentinos = equiposOlimpicosrdd.filter(lambda l: "MEX" in l)
equipos_argentinos.collect()

[['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['193', 'Chamukina', 'MEX'],
 ['656', 'Mexico', 'MEX'],
 ['657', 'Mexico-1', 'MEX'],
 ['658', 'Mexico-2', 'MEX'],
 ['700', 'Nausikaa 4', 'MEX'],
 ['1053', 'Tlaloc', 'MEX'],
 ['1168', 'Xolotl', 'MEX']]

In [12]:
equiposOlimpicosrdd.count()

1185

In [13]:
equiposOlimpicosrdd.countApprox(20)

1185

In [3]:
! ls ../files

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [15]:
deportistaOlimpicoRDD = sc.textFile("../files/deportista.csv") \
                        .map(lambda l: l.split(","))
deportistaOlimpicoRDD2 = sc.textFile("../files/deportista2.csv") \
                        .map(lambda l: l.split(","))

In [16]:
deportistaOlimpicoRDD=deportistaOlimpicoRDD \
                      .union(deportistaOlimpicoRDD2)

In [19]:
deportistaOlimpicoRDD.count()

135572

In [20]:
equiposOlimpicosrdd.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [21]:
deportistaOlimpicoRDD.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

In [24]:
deportistaOlimpicoRDD.map(lambda l: [l[-1], l[:-1]]) \
                    .join(equiposOlimpicosrdd).map(lambda x : [x[0], x[-1]]) \
                    .take(6)

[['199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'China')],
 ['199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'China')],
 ['199', (['602', 'Abudoureheman', '1', '22', '182', '75'], 'China')],
 ['199', (['1463', 'Ai Linuer', '1', '25', '160', '62'], 'China')],
 ['199', (['1464', 'Ai Yanhan', '2', '14', '168', '54'], 'China')],
 ['199', (['3605', 'An Weijiang', '1', '22', '178', '72'], 'China')]]

In [39]:
deportistaJoin =  deportistaOlimpicoRDD.map(lambda l: [l[-1], l[:-1]]) \
                    .join(equiposOlimpicosrdd).map(lambda x : [x[0], x[-1]])


In [43]:
print(deportistaJoin)

[['507', (['73449', 'Mario Giovanni Marcello Magnozzi', '1', '22', '0', '0'], 'Italy')], ['874', (['16499', 'Pavel Vladimirovich Bure', '1', '26', '180', '86'], 'Russia')], ['174', (['106127', 'Graeme Saunders', '1', '26', '176', '64'], 'Canada')], ['362', (['85410', 'Claude Netter', '1', '27', '177', '75'], 'France')], ['816', (['38243', 'Vtor Manuel Gomes Gamito', '1', '30', '183', '66'], 'Portugal')], ['1019', (['37680', 'Philippe Furrer', '1', '24', '187', '93'], 'Switzerland')]]


In [31]:
resultadosOlimpicosRDD = sc.textFile("../files/resultados.csv").map(lambda l: l.split(","))
resultadosOlimpicosRDD.take(2)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['1', 'NA', '1', '39', '1']]

In [32]:
resultadosOlimpicosRDD = resultadosOlimpicosRDD.filter(lambda l: 'NA' not in l[1])
resultadosOlimpicosRDD.take(2)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4']]

In [48]:
equiposOlimpicosrdd.map(lambda l: [l[0], l[:-1]]) \
                 .join(deportistaOlimpicoRDD).map(lambda x : [x[-1], x[:-1]]) \
                 .take(1)
                                         

[[(['2', 'A North American Team'], 'A Lamusi'), ('2',)]]

In [63]:
 deportistaOlimpicoRDD.map(lambda l: [l[-1], l[:-1]]) \
                    .join(equiposOlimpicosrdd.map(lambda x : [x[0], x[2]])) \
                    .take(2)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN'))]

In [67]:
 deportistaOlimpicoRDD.map(lambda l: [l[-1], l[:-1]]) \
                    .join(equiposOlimpicosrdd.map(lambda x : [x[0], x[2]])) \
                    .map(lambda y: [y[1][0][0], y[1][0][1]]) \
                    .join(resultadosOlimpicosRDD.map(lambda z: [z[2], z[1]])) \
                    .take(6)

[('7597', ('Bao Yingying', 'Silver')),
 ('17282', ('Cai Huijue', 'Bronze')),
 ('17996', ('Cao Mianying', 'Silver')),
 ('19779', ('Chang Si', 'Silver')),
 ('19791', ('Chang Yongxiang', 'Silver')),
 ('20215', ('Chen Jin', 'Bronze'))]